**Sınıflar - Classes**

https://goalkicker.com/PythonBook/PythonNotesForProfessionals.pdf s 209

Python, kendisini yalnızca popüler bir betik dili olarak sunmakla kalmaz, aynı zamanda nesne yönelimli programlama paradigmasını da destekler. Sınıflar, verileri tanımlar ve bu verileri işlemek için yöntemler sağlar, hepsi tek bir nesne altında kapsanır. Dahası, sınıflar somut uygulama ayrıntılarını verilerin soyut temsillerinden ayırarak soyutlamaya izin verir.

Sınıfları kullanan kod genellikle okunması, anlaşılması ve sürdürülmesi daha kolaydır.


**Introduction to classes**

Bir sınıf, belirli bir nesnenin temel özelliklerini tanımlayan bir şablon işlevi görür. İşte bir örnek:

In [62]:
class Person(object):
    """A simple class."""        # docstring
    species = "Homo Sapiens"     # class attribute

    def __init__(self, name):    # special method
        """This is the initializer. It's a special
       method (see below).
        """
        self.name = name             # instance attribute

    def __str__(self):           # special method
        """This method is run when Python tries
    to cast the object to a string. Return
    this string when using print(), etc.
        """
        return self.name

    def rename(self, renamed):   # regular method
        """Reassign and print the name attribute."""
        self.name = renamed
        print("Now my name is {}".format(self.name))

There are a few things to note when looking at the above example.
1. The class is made up of attributes (data) and methods (functions).
2. Attributes and methods are simply defined as normal variables and functions.
3. As noted in the corresponding docstring, the  __ init__() method is called the initializer. It's equivalent to the
constructor in other object oriented languages, and is the method that is first run when you create a new
object, or new instance of the class.
4. Attributes that apply to the whole class are defined first, and are called class attributes.
5. Attributes that apply to a specific instance of a class (an object) are called instance attributes. They are
generally defined inside __init__(); this is not necessary, but it is recommended (since attributes defined
outside of __init__() run the risk of being accessed before they are defined).
6. Every method, included in the class definition passes the object in question as its first parameter. The word
self is used for this parameter (usage of self is actually by convention, as the word self has no inherent
meaning in Python, but this is one of Python's most respected conventions, and you should always follow it).
7. Those used to object-oriented programming in other languages may be surprised by a few things. One is that
Python has no real concept of private elements, so everything, by default, imitates the behavior of the
C++/Java public keyword. For more information, see the "Private Class Members" example on this page.
8. Some of the class's methods have the following form: __functionname__(self, other_stuff). All such
methods are called "magic methods" and are an important part of classes in Python. For instance, operator
overloading in Python is implemented with magic methods. For more information, see the relevant documentation.


In [63]:
# Now let's make a few instances of our Person class!

# Instances
kelly = Person("Kelly")
joseph = Person("Joseph")
john_doe = Person("John Doe")
#We currently have three Person objects, kelly, joseph, and john_doe.

In [64]:
# We can access the attributes of the class from each instance using the dot operator . Note again the difference between class and instance attributes:

# Attributes
kelly.species
john_doe.species
joseph.species
kelly.name
joseph.name

'Joseph'

In [ ]:
# We can execute the methods of the class using the same dot operator .:

# Methods
john_doe.__str__()
print(john_doe)
john_doe.rename("John")

**Bound, unbound, and static methods**

The idea of bound and unbound methods was removed in Python 3. In Python 3 when you declare a method within
a class, you are using a def keyword, thus creating a function object. This is a regular function, and the surrounding
class works as its namespace. In the following example we declare method f within class A, and it becomes a
function A.f:

In [ ]:
class A(object):
    def f(self, x):
        return 2 * x
A.f
# <function A.f at ...> (in Python 3.x)

In Python 2 the behavior was different: function objects within the class were implicitly replaced with objects of type instancemethod, which were called unbound methods because they were not bound to any particular class instance.
It was possible to access the underlying function using ._ _func_ _ property.

In [ ]:
A.f
# <unbound method A.f> (in Python 2.x)
A.f.__class__
# <type 'instancemethod'>
A.f.__func__
# <function f at ...>

The latter behaviors are confirmed by inspection - methods are recognized as functions in Python 3, while the distinction is upheld in Python 2.

In [ ]:
import inspect
inspect.isfunction(A.f)
# True
inspect.ismethod(A.f)
# False

In [ ]:
# Python 2.x Version ≥ 2.3
import inspect
inspect.isfunction(A.f)
# False
inspect.ismethod(A.f)
# True

In both versions of Python function/method A.f can be called directly, provided that you pass an instance of class A as the first argument.


In [ ]:
A.f(1, 7)
# Python 2: TypeError: unbound method f() must be called with
# A instance as first argument (got int instance instead)
# Python 3: 14
a = A()
A.f(a, 20)
# Python 2 & 3: 40

Now suppose a is an instance of class A, what is a.f then? Well, intuitively this should be the same method f of class A, only it should somehow "know" that it was applied to the object a – in Python this is called method bound to a.

The nitty-gritty details are as follows: writing a.f invokes the magic _ _ getattribute_ _ method of a, which first checks whether a has an attribute named f (it doesn't), then checks the class A whether it contains a method with
such a name (it does), and creates a new object m of type method which has the reference to the original A.f in m. _ _ func _ _ , and a reference to the object a in m. _ _self _ _. When this object is called as a function, it simply does
the following: m(...) => m. _ _func _ _ (m. _ _ self _ _ , ...). Thus this object is called a bound method because when invoked it knows to supply the object it was bound to as the first argument. (These things work same way in Python 2 and 3).

In [ ]:
a = A()
a.f
# <bound method A.f of <__main__.A object at ...>>
a.f(2)
# 4
# Note: the bound method object a.f is recreated *every time* you call it:
a.f is a.f # False
# As a performance optimization you can store the bound method in the object's
# __dict__, in which case the method object will remain fixed:
a.f = a.f

Finally, Python has class methods and static methods – special kinds of methods. Class methods work the same way as regular methods, except that when invoked on an object they bind to the class of the object instead of to the
object. Thus m. _ _ self _ _ = type(a). When you call such bound method, it passes the class of a as the first argument. Static methods are even simpler: they don't bind anything at all, and simply return the underlying function without any transformations.

In [ ]:
class D(object):
    multiplier = 2

    @classmethod
    def f(cls, x):
        return cls.multiplier * x

    @staticmethod
    def g(name):
        print("Hello, %s" % name)

D.f
# <bound method type.f of <class '__main__.D'>>
D.f(12)
# 24
D.g
# <function D.g at ...>
D.g("world")
# Hello, world


In [ ]:
# Note that class methods are bound to the class even when accessed on the instance:

d = D()
d.multiplier = 1337
(D.multiplier, d.multiplier)
# (2, 1337)
d.f
# <bound method D.f of <class '__main__.D'>>
d.f(10)
# 20


It is worth noting that at the lowest level, functions, methods, staticmethods, etc. are actually descriptors that invoke _ _ get _ _ , _ _ set _ _ and optionally __ del__ special methods. For more details on classmethods and
staticmethods:

What is the difference between @staticmethod and @classmethod in Python?
https://stackoverflow.com/questions/136097/what-is-the-difference-between-staticmethod-and-classmethod-in-python

Meaning of @classmethod and @staticmethod for beginner?
https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner